## support vector machine con grid search

In [14]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


#setting up labels for dataset
labels = ['class', 'spec_number', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column
#display(df)
#separating y from x
X = df.iloc[:, 1:16]
y = df.iloc[:, 0]

display(X)

,spec_number,eccentr,asp_ratio,elong,solidity,stoch_conv,iso_factor,max_ind_depth,lobedness,av_intensity,av_contr,smooth,third_mom,unif,entropy
0,3,0.84579,2.0052,0.49931,0.96585,0.99825,0.65149,0.019981,0.072660,0.005034,0.036189,0.001308,0.000351,0.000009,0.22547
1,1,0.99571,10.3520,0.90609,0.90652,0.97895,0.16732,0.025787,0.121030,0.023523,0.089455,0.007939,0.002888,0.000069,0.73774
2,3,0.61289,1.0991,0.33117,0.92405,0.98421,0.61661,0.025545,0.118770,0.054687,0.160600,0.025145,0.011672,0.000121,1.10760
3,2,0.91186,2.4994,0.60323,0.98300,1.00000,0.64916,0.006149,0.006882,0.018887,0.072486,0.005227,0.001489,0.000083,0.67811
4,8,0.93613,2.3898,0.67175,0.84434,0.90526,0.37007,0.039277,0.280770,0.048447,0.119550,0.014091,0.003789,0.000425,1.25930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,1,0.71763,1.5041,0.33723,0.98803,1.00000,0.82342,0.004042,0.002974,0.050064,0.125420,0.015486,0.004663,0.000271,1.35980
336,5,0.88566,2.2029,0.54993,0.98635,1.00000,0.59039,0.003760,0.002573,0.039469,0.124590,0.015285,0.005660,0.000204,0.84271
337,9,0.86988,2.1277,0.53926,0.96257,1.00000,0.64927,0.027899,0.141660,0.026849,0.096968,0.009315,0.003238,0.000104,0.72710
338,10,0.86675,2.0417,0.52999,0.94477,0.98947,0.55141,0.028749,0.150430,0.031023,0.120070,0.014211,0.006454,0.000069,0.64966


#### grid search con k-fold CV e LOOCV

domande:

- cos'è rbf (in kernel)?

- il rescaling delle variabili come si fa (probabilmente lo fa automaticamente)?

- C è la tolleranza (in particolare corrisponde alla prima o alla seconda formulazione di Medvet)?

- perché roc_auc_ovo e roc_auc_ovr non funzionano?

- cercare modo migliore per 'elencare' i regularization parameter

domande generali:

- ha senso tenere spec_number nel dataset?

- come si fa a mettere più di un effectivness index nella grid search?

In [17]:
k = 5

# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_cv = GridSearchCV(svm.SVC(), grid_param, cv=k, scoring='accuracy')
clf_cv.fit(X, y)
print(clf_cv.best_params_)
print(clf_cv.best_score_)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01,
       1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07]),
                         'decision_function_shape': ['ovo', 'ovr'],
                         'degree': array([2, 3, 4]),
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

{'C': 100000.0, 'decision_function_shape': 'ovo', 'degree': 2, 'kernel': 'linear'}
0.7058823529411764


In [ ]:
# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_loocv = GridSearchCV(svm.SVC(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
clf_loocv.fit(X, y)
print(clf_loocv.best_params_)
print(clf_loocv.best_score_)